In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install wandb -qU
!wandb login

     |████████████████████████████████| 1.9 MB 14.9 MB/s 
     |████████████████████████████████| 166 kB 58.6 MB/s 
     |████████████████████████████████| 182 kB 70.6 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 166 kB 71.5 MB/s 
     |████████████████████████████████| 162 kB 70.3 MB/s 
     |████████████████████████████████| 162 kB 65.6 MB/s 
     |████████████████████████████████| 158 kB 73.0 MB/s 
     |████████████████████████████████| 157 kB 67.4 MB/s 
     |████████████████████████████████| 157 kB 71.8 MB/s 
     |████████████████████████████████| 157 kB 73.8 MB/s 
     |████████████████████████████████| 157 kB 72.9 MB/s 
     |████████████████████████████████| 157 kB 77.0 MB/s 
     |████████████████████████████████| 157 kB 74.2 MB/s 
     |████████████████████████████████| 157 kB 75.9 MB/s 
     |████████████████████████████████| 156 kB 72.5 MB/s 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import wandb
from wandb.keras import WandbCallback

In [4]:
tf.keras.utils.set_random_seed(1)
tf.config.experimental.enable_op_determinism()

In [5]:
PATH = '/content/drive/MyDrive/Research Internship/final data/wear particles/Train'
test_path = '/content/drive/MyDrive/Research Internship/final data/wear particles/Test'
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

test_ds = tf.keras.utils.image_dataset_from_directory(
  test_path,
  seed = 42,
  image_size=(img_height, img_width))

Found 879 files belonging to 5 classes.
Using 704 files for training.
Found 879 files belonging to 5 classes.
Using 175 files for validation.
Found 94 files belonging to 5 classes.


In [6]:
class_names = train_ds.class_names
test_class_names = test_ds.class_names
num_classes = len(class_names)
print(class_names)

['cutting wear', 'normal rubbing and cutting wear combined', 'normal rubbing wear', 'red oxide', 'severe sliding wear']


In [23]:
resnet50 = tf.keras.applications.ResNet50(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling='avg')
model_resnet50=Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  resnet50,
  #layers.Flatten(),
  #layers.Dense(32, activation='relu'),
  layers.Dropout(0.2), 
  layers.Dense(num_classes, activation='softmax')
])
resnet50.trainable=False
model_resnet50.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_4 (Rescaling)     (None, 224, 224, 3)       0         
                                                                 
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dropout_4 (Dropout)         (None, 2048)              0         
                                                                 
 dense_6 (Dense)             (None, 5)                 10245     
                                                                 
Total params: 23,597,957
Trainable params: 10,245
Non-trainable params: 23,587,712
_________________________________________________________________


In [24]:
model_resnet50.compile(
              optimizer= tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam'),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy']
              )

In [25]:
wandb.init(
      # Set the project where this run will be logged
      project="Transfer_Learning_New", 
      name = "resnet50",
)

In [26]:
%%time
epochs=40
history = model_resnet50.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
callbacks=[WandbCallback()]
)

Epoch 1/40
22/22 [==============================] - ETA: 0s - loss: 1.5272 - accuracy: 0.3651

wandb: Adding directory to artifact (/content/wandb/run-20221104_134618-2z9le3jh/files/model-best)... Done. 0.8s


22/22 [==============================] - 33s 1s/step - loss: 1.5272 - accuracy: 0.3651 - val_loss: 1.3641 - val_accuracy: 0.3943
Epoch 2/40
22/22 [==============================] - ETA: 0s - loss: 1.3385 - accuracy: 0.4801

wandb: Adding directory to artifact (/content/wandb/run-20221104_134618-2z9le3jh/files/model-best)... Done. 0.8s


22/22 [==============================] - 26s 1s/step - loss: 1.3385 - accuracy: 0.4801 - val_loss: 1.2814 - val_accuracy: 0.5314
Epoch 3/40
22/22 [==============================] - ETA: 0s - loss: 1.2672 - accuracy: 0.5369

...

22/22 [==============================] - 26s 1s/step - loss: 0.9262 - accuracy: 0.6449 - val_loss: 0.9609 - val_accuracy: 0.5714
Epoch 37/40
22/22 [==============================] - 4s 167ms/step - loss: 0.9281 - accuracy: 0.6264 - val_loss: 0.9726 - val_accuracy: 0.5714
Epoch 38/40
22/22 [==============================] - ETA: 0s - loss: 0.9244 - accuracy: 0.6364

wandb: Adding directory to artifact (/content/wandb/run-20221104_134618-2z9le3jh/files/model-best)... Done. 0.8s


22/22 [==============================] - 26s 1s/step - loss: 0.9244 - accuracy: 0.6364 - val_loss: 0.9469 - val_accuracy: 0.5714
Epoch 39/40
22/22 [==============================] - 4s 168ms/step - loss: 0.9168 - accuracy: 0.6406 - val_loss: 0.9605 - val_accuracy: 0.5543
Epoch 40/40
22/22 [==============================] - ETA: 0s - loss: 0.9156 - accuracy: 0.6562

wandb: Adding directory to artifact (/content/wandb/run-20221104_134618-2z9le3jh/files/model-best)... Done. 0.8s


22/22 [==============================] - 26s 1s/step - loss: 0.9156 - accuracy: 0.6562 - val_loss: 0.9418 - val_accuracy: 0.5771
CPU times: user 9min 49s, sys: 42 s, total: 10min 31s
Wall time: 14min 33s


In [19]:
wandb.finish()

In [20]:
!nvidia-smi --query-gpu=gpu_name --format=csv


name
Tesla T4
